In [1]:
from datetime import datetime

from pyspark import SparkContext, SQLContext
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, TimestampType, DoubleType, StringType, IntegerType

from sparkts.datetimeindex import uniform, BusinessDayFrequency
from sparkts.timeseriesrdd import time_series_rdd_from_observations

In [2]:
import sparkts.datetimeindex as dt

In [3]:
dt.DayFrequency

sparkts.datetimeindex.DayFrequency

In [4]:
def lineToRow(line):
    (year, month, day, symbol, volume, price) = line.split("\t")
    # Python 2.x compatible timestamp generation
    dt = datetime(int(year), int(month), int(day))
    return (dt, symbol, float(price))

def loadObservations(sparkContext, sqlContext, path):
    textFile = sparkContext.textFile(path)
    rowRdd = textFile.map(lineToRow)
    schema = StructType([
        StructField('timestamp', TimestampType(), nullable=True),
        StructField('symbol', StringType(), nullable=True),
        StructField('price', DoubleType(), nullable=True),
    ])
    return sqlContext.createDataFrame(rowRdd, schema);

In [3]:
!wget https://raw.githubusercontent.com/sryza/spark-ts-examples/master/data/ticker.tsv

--2016-10-06 17:14:18--  https://raw.githubusercontent.com/sryza/spark-ts-examples/master/data/ticker.tsv
Resolving raw.githubusercontent.com... 151.101.60.133
Connecting to raw.githubusercontent.com|151.101.60.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105146 (103K) [text/plain]
Saving to: 'ticker.tsv.1'

ticker.tsv.1        100%[=====================>] 102.68K  --.-KB/s   in 0.1s   

2016-10-06 17:14:19 (719 KB/s) - 'ticker.tsv.1' saved [105146/105146]



In [4]:
tickerObs = loadObservations(sc, sqlContext, "/Users/guillermobreto/Downloads/spark-timeseries/DOCS_REPO/spark-timeseries/ticker.tsv")

In [24]:
tickerObs.select("timestamp").take(3)

[Row(timestamp=datetime.datetime(2015, 8, 14, 0, 0)),
 Row(timestamp=datetime.datetime(2015, 9, 14, 0, 0)),
 Row(timestamp=datetime.datetime(2015, 9, 18, 0, 0))]

In [252]:
tickerObs.show(3, truncate=False)

+---------------------+------+------+
|timestamp            |symbol|price |
+---------------------+------+------+
|2015-08-14 00:00:00.0|ADP   |82.99 |
|2015-09-14 00:00:00.0|NKE   |111.78|
|2015-09-18 00:00:00.0|DO    |20.18 |
+---------------------+------+------+
only showing top 3 rows



In [29]:
tickerObs.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)



In [26]:

# Create an daily DateTimeIndex over August and September 2015
freq = BusinessDayFrequency(1, 1, sc)
dtIndex = uniform(start='2015-08-03T00:00-04:00', end='2015-09-22T00:00-04:00', freq=freq, sc=sc)



In [27]:
tickerTsrdd = time_series_rdd_from_observations(dtIndex, tickerObs, "timestamp", "symbol", "price")


In [28]:
tickerTsrdd.take(2)

[(u'AAL',
  array([ 41.71,  42.74,  42.96,  42.18,  41.49,  41.68,  42.7 ,  42.52,
          42.67,  42.88,  43.95,  43.63,  43.53,  41.98,  39.75,  37.62,
          37.5 ,  38.85,  39.03,  38.62,  38.98,  39.2 ,  41.51,  40.89,
          40.6 ,    nan,  40.8 ,  41.  ,  41.21,  42.15,  42.29,  42.64,
          43.06,  43.99,  43.49,  43.24,  41.19])),
 (u'AAPL',
  array([ 118.44  ,  114.64  ,  115.4   ,  115.13  ,  115.52  ,  119.6901,
          113.5499,  115.24  ,  115.15  ,  116.    ,  117.1601,  116.5   ,
          115.01  ,  112.65  ,  105.76  ,  103.155 ,  103.74  ,  109.625 ,
          112.92  ,  113.29  ,  112.76  ,  107.72  ,  112.34  ,  110.37  ,
          109.27  ,       nan,  112.21  ,  110.15  ,  112.57  ,  114.017 ,
          115.3   ,  116.28  ,  116.35  ,  113.92  ,  113.45  ,  115.23  ,
          113.432 ]))]

In [11]:

# Count the number of series (number of symbols)
print(tickerTsrdd.count())

# Impute missing values using linear interpolation
filled = tickerTsrdd.fill("linear")

# Compute return rates
returnRates = filled.return_rates()

104


In [12]:
filled.take(2)

[(u'AAL',
  array([ 41.71,  42.74,  42.96,  42.18,  41.49,  41.68,  42.7 ,  42.52,
          42.67,  42.88,  43.95,  43.63,  43.53,  41.98,  39.75,  37.62,
          37.5 ,  38.85,  39.03,  38.62,  38.98,  39.2 ,  41.51,  40.89,
          40.6 ,  40.7 ,  40.8 ,  41.  ,  41.21,  42.15,  42.29,  42.64,
          43.06,  43.99,  43.49,  43.24,  41.19])),
 (u'AAPL',
  array([ 118.44  ,  114.64  ,  115.4   ,  115.13  ,  115.52  ,  119.6901,
          113.5499,  115.24  ,  115.15  ,  116.    ,  117.1601,  116.5   ,
          115.01  ,  112.65  ,  105.76  ,  103.155 ,  103.74  ,  109.625 ,
          112.92  ,  113.29  ,  112.76  ,  107.72  ,  112.34  ,  110.37  ,
          109.27  ,  110.74  ,  112.21  ,  110.15  ,  112.57  ,  114.017 ,
          115.3   ,  116.28  ,  116.35  ,  113.92  ,  113.45  ,  115.23  ,
          113.432 ]))]

In [12]:
# Durbin-Watson test for serial correlation, ported from TimeSeriesStatisticalTests.scala
def dwtest(residuals):
    residsSum = residuals[0] * residuals[0]
    diffsSum = 0.0
    i = 1
    while i < len(residuals):
        residsSum += residuals[i] * residuals[i]
        diff = residuals[i] - residuals[i - 1]
        diffsSum += diff * diff
        i += 1
    return diffsSum / residsSum

# Compute Durbin-Watson stats for each series
# Swap ticker symbol and stats so min and max compare the statistic value, not the
# ticker names.
dwStats = returnRates.map_series(lambda row: (row[0], [dwtest(row[1])])).map(lambda x: (x[1], x[0]))

print(dwStats.min())
print(dwStats.max())

([0.99930539174187916], u'NFLX')
([2.3701164736953166], u'DISCK')


# Get the data

In [5]:
rdd = sc.wholeTextFiles("/Users/guillermobreto/Downloads/fred_timeseries_project/data/fred_codes/")
print(rdd.count())
from pyspark.sql.functions import explode
import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from datetime import datetime

from pyspark import SparkContext, SQLContext
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, TimestampType, DoubleType, StringType

from sparkts.datetimeindex import uniform, BusinessDayFrequency
from sparkts.timeseriesrdd import time_series_rdd_from_observations

import numpy as np
import pandas as pd

40809


In [12]:
from sparkts.datetimeindex import DayFrequency

In [13]:
freq = DayFrequency(1,sc)

In [14]:
%matplotlib inline 
import matplotlib.pyplot as plt

In [15]:
#freq = BusinessDayFrequency(1, 1, sc)
dtIndex = uniform(start='2005-02-01T00:00-05:00', end='2005-06-01T00:00-05:00', freq=freq, sc=sc)

In [16]:
rdd_df = rdd.map(lambda r: (r[0].split("/")[-1].strip(".csv"),filter(None, r[1].split("\n")[1:]))).toDF(["symbol","v"])


In [11]:
rdd_df.select("symbol").distinct().count()

40809

In [17]:
rdd_df=rdd_df.limit(100)
rdd_df.show(3)

+--------------------+--------------------+
|              symbol|                   v|
+--------------------+--------------------+
|FRED_00XALCCHM086...|[2005-01-01,97.0,...|
|FRED_00XALCFIM086...|[1996-01-01,71.22...|
|FRED_00XALCHRM086...|[2004-12-01,81.32...|
+--------------------+--------------------+
only showing top 3 rows



In [18]:
rdd_df.select("symbol").distinct().count()

100

In [19]:
rdd_df_exp =  rdd_df.select([rdd_df.symbol,explode(rdd_df.v).alias("DATA-VALUE")])

In [20]:
valueUdf = udf(lambda s: float(s.split(",")[1]), DoubleType())
dateUdf = udf(lambda s: s.split(",")[0], StringType())
new_df =rdd_df_exp.withColumn("Date", (f.to_date(f.lit(dateUdf(rdd_df_exp["DATA-VALUE"]))).cast(TimestampType())))
new_df =new_df.withColumn("price", valueUdf(new_df["DATA-VALUE"]))

In [21]:
new_df.show(3)

+--------------------+---------------+--------------------+-----+
|              symbol|     DATA-VALUE|                Date|price|
+--------------------+---------------+--------------------+-----+
|FRED_00XALCCHM086...|2005-01-01,97.0|2005-01-01 00:00:...| 97.0|
|FRED_00XALCCHM086...|2005-02-01,97.3|2005-02-01 00:00:...| 97.3|
|FRED_00XALCCHM086...|2005-03-01,97.4|2005-03-01 00:00:...| 97.4|
+--------------------+---------------+--------------------+-----+
only showing top 3 rows



In [22]:
new_df.select("symbol").distinct().count()

100

In [23]:
freq = DayFrequency(1,sc)
dtIndex = uniform(start='2015-01-01T00:00-05:00', end='2016-10-01T00:00-05:00', freq=freq, sc=sc)

In [24]:
dates = ("2015-01-01",  "2016-10-01")
date_from, date_to = [f.to_date(f.lit(s)).cast(TimestampType()) for s in dates]
df_filtered = new_df.where((new_df.Date > date_from) & (new_df.Date < date_to))

In [25]:
df_filtered.show(3)

+--------------------+-----------------+--------------------+------+
|              symbol|       DATA-VALUE|                Date| price|
+--------------------+-----------------+--------------------+------+
|FRED_00XALCCHM086...| 2015-02-01,99.93|2015-02-01 00:00:...| 99.93|
|FRED_00XALCCHM086...| 2015-03-01,100.4|2015-03-01 00:00:...| 100.4|
|FRED_00XALCCHM086...|2015-04-01,100.13|2015-04-01 00:00:...|100.13|
+--------------------+-----------------+--------------------+------+
only showing top 3 rows



In [26]:
df = df_filtered.select(["symbol", "Date", "price"])
df = df.withColumnRenamed("Date", "timestamp")

In [27]:
df.show(2, truncate=False)

+---------------------+---------------------+-----+
|symbol               |timestamp            |price|
+---------------------+---------------------+-----+
|FRED_00XALCCHM086NEST|2015-02-01 00:00:00.0|99.93|
|FRED_00XALCCHM086NEST|2015-03-01 00:00:00.0|100.4|
+---------------------+---------------------+-----+
only showing top 2 rows



In [28]:
tickerTsrdd = time_series_rdd_from_observations(dtIndex, df, "timestamp", "symbol", "price")

In [29]:
tickerTsrdd.take(3)

[(u'FRED_00XALCCHM086NEST',
  array([    nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,   99.93,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,  100.4 ,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,  100.13,
             nan,     nan,     nan, 

In [30]:
filled = tickerTsrdd.fill("linear")

In [31]:
filled.take(2)

[(u'FRED_00XALCCHM086NEST',
  array([          nan,           nan,           nan,           nan,
                   nan,           nan,           nan,           nan,
                   nan,           nan,           nan,           nan,
                   nan,           nan,           nan,           nan,
                   nan,           nan,           nan,           nan,
                   nan,           nan,           nan,           nan,
                   nan,           nan,           nan,           nan,
                   nan,           nan,           nan,   99.93      ,
           99.94678571,   99.96357143,   99.98035714,   99.99714286,
          100.01392857,  100.03071429,  100.0475    ,  100.06428571,
          100.08107143,  100.09785714,  100.11464286,  100.13142857,
          100.14821429,  100.165     ,  100.18178571,  100.19857143,
          100.21535714,  100.23214286,  100.24892857,  100.26571429,
          100.2825    ,  100.29928571,  100.31607143,  100.33285714,
      

In [32]:
previous = filled.fill("previous")

In [33]:
previous.take(3)

[(u'FRED_00XALCCHM086NEST',
  array([          nan,           nan,           nan,           nan,
                   nan,           nan,           nan,           nan,
                   nan,           nan,           nan,           nan,
                   nan,           nan,           nan,           nan,
                   nan,           nan,           nan,           nan,
                   nan,           nan,           nan,           nan,
                   nan,           nan,           nan,           nan,
                   nan,           nan,           nan,   99.93      ,
           99.94678571,   99.96357143,   99.98035714,   99.99714286,
          100.01392857,  100.03071429,  100.0475    ,  100.06428571,
          100.08107143,  100.09785714,  100.11464286,  100.13142857,
          100.14821429,  100.165     ,  100.18178571,  100.19857143,
          100.21535714,  100.23214286,  100.24892857,  100.26571429,
          100.2825    ,  100.29928571,  100.31607143,  100.33285714,
      

In [34]:
nearest = previous.fill("nearest")

In [35]:
nearest.take(1)

[(u'FRED_00XALCCHM086NEST',
  array([          nan,   99.93      ,   99.93      ,   99.93      ,
           99.93      ,   99.93      ,   99.93      ,   99.93      ,
           99.93      ,   99.93      ,   99.93      ,   99.93      ,
           99.93      ,   99.93      ,   99.93      ,   99.93      ,
           99.93      ,   99.93      ,   99.93      ,   99.93      ,
           99.93      ,   99.93      ,   99.93      ,   99.93      ,
           99.93      ,   99.93      ,   99.93      ,   99.93      ,
           99.93      ,   99.93      ,   99.93      ,   99.93      ,
           99.94678571,   99.96357143,   99.98035714,   99.99714286,
          100.01392857,  100.03071429,  100.0475    ,  100.06428571,
          100.08107143,  100.09785714,  100.11464286,  100.13142857,
          100.14821429,  100.165     ,  100.18178571,  100.19857143,
          100.21535714,  100.23214286,  100.24892857,  100.26571429,
          100.2825    ,  100.29928571,  100.31607143,  100.33285714,
      

In [36]:
rr = nearest.return_rates()

In [37]:
rr = rr.fill("linear")

In [38]:
rr = rr.map(lambda ts: (ts[0], np.nan_to_num(ts[1])))

In [39]:
rr.take(1)

[(u'FRED_00XALCCHM086NEST',
  array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   1.67974725e-04,   1.67946514e-04,
           1.67918313e-04,   1.67890121e-04,   1.67861939e-04,
           1.67833766e-04,   1.67805603e-04,   1.67777449e-04,
           1.67749304e-04,   1.67721169e-04,   1.67693043e-04,
           1.67664927e-04,   1.67636820e-04,   1.67608723e-04,
           1.67580635e-04, 

In [40]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n


def shifting(a, delta):
    from scipy.ndimage.interpolation import shift
    return shift(a, delta, cval=np.NaN)

In [41]:
ma = rr.map(lambda row:  (row[0], moving_average(row[1], 10)))

In [42]:
ma.take(2)

[(u'FRED_00XALCCHM086NEST',
  array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   1.67974725e-05,   3.35921240e-05,
           5.03839553e-05,   6.71729674e-05,   8.39591613e-05,
           1.00742538e-04,   1.17523098e-04,   1.34300843e-04,
           1.51075773e-04,   1.67847890e-04,   1.67819722e-04,
           1.67791563e-04,   1.67763414e-04,   1.67735274e-04,
           1.67707144e-04,   1.67679023e-04,   1.67650911e-04,
           1.67622809e-04,   1.67594716e-04,   1.67566633e-04,
           1.67538559e-04,   1.67510495e-04,   1.67482440e-04,
           1.67454394e-04, 

In [43]:
sh = rr.map(lambda row:  (row[0] + '_shift', np.nan_to_num(shifting(row[1], 1))))
ma = rr.map(lambda row:  (row[0] + "_mov_avg", moving_average(row[1])))

In [44]:
sh.take(1)

[(u'FRED_00XALCCHM086NEST_shift',
  array([  0.00000000e+00,   2.46481046e-22,   3.76158192e-37,
          -1.50463277e-36,   3.00926554e-36,  -2.40741243e-35,
           4.81482486e-35,  -2.88889492e-34,   7.70371978e-34,
          -3.08148791e-33,   1.23259516e-32,  -4.93038066e-32,
           1.97215226e-31,  -3.94430453e-31,   3.15544362e-30,
          -1.26217745e-29,   3.78653235e-29,  -1.00974196e-28,
           4.03896783e-28,  -2.42338070e-27,   6.46234854e-27,
          -2.58493941e-26,   1.03397577e-25,  -4.13590306e-25,
           8.27180613e-25,  -6.61744490e-24,   1.32348898e-23,
          -7.94093388e-23,   3.17637355e-22,  -1.27054942e-21,
           5.08219768e-21,  -6.77626358e-21,   1.67974725e-04,
           1.67946514e-04,   1.67918313e-04,   1.67890121e-04,
           1.67861939e-04,   1.67833766e-04,   1.67805603e-04,
           1.67777449e-04,   1.67749304e-04,   1.67721169e-04,
           1.67693043e-04,   1.67664927e-04,   1.67636820e-04,
           1.67608723

In [45]:
total = sc.union([rr, ma, sh])

In [46]:
lenghts = total.map(lambda ts: len(ts[1]))

In [47]:
colLength = np.array(lenghts.collect())

In [48]:
colLength.min()

637

In [49]:
total.count()

87

In [50]:
from pyspark.mllib.linalg import Vectors
total_df = total.map(lambda x: Row(symbol=x[0], feat=Vectors.dense(x[1]))).map(lambda x: [x[1], x[0]]).toDF(["symbol","feat"])

In [51]:
total_df.show(3)

+--------------------+--------------------+
|              symbol|                feat|
+--------------------+--------------------+
|FRED_00XALCCHM086...|[0.0,0.0,0.0,0.0,...|
|FRED_00XALCFIM086...|[0.0,0.0,0.0,0.0,...|
|FRED_00XALCHRM086...|[0.0,0.0,0.0,0.0,...|
+--------------------+--------------------+
only showing top 3 rows



In [52]:
udfToArray = udf(lambda s: len(s), IntegerType())

In [53]:
total_df.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- feat: vector (nullable = true)



In [54]:
total_df= total_df.withColumn("length",udfToArray(total_df.feat) )

In [55]:
total_df.show(2)

+--------------------+--------------------+------+
|              symbol|                feat|length|
+--------------------+--------------------+------+
|FRED_00XALCCHM086...|[0.0,0.0,0.0,0.0,...|   639|
|FRED_00XALCFIM086...|[0.0,0.0,0.0,0.0,...|   639|
+--------------------+--------------------+------+
only showing top 2 rows



In [56]:
total_df_clean = total_df.filter("length=639")

In [57]:
limited =  total_df_clean.map(lambda ts: [ts[0], filter(None,[float(l) for l in ts[1].toArray()])])

In [58]:
ts = limited.toDF(["Symbol", "ts"])

In [59]:
ts.show(1)

+--------------------+--------------------+
|              Symbol|                  ts|
+--------------------+--------------------+
|FRED_00XALCCHM086...|[1.67974725164832...|
+--------------------+--------------------+
only showing top 1 row



In [60]:
ts_exploded = ts.select([ts.Symbol,explode(ts.ts).alias("values")])

In [61]:
ts_exploded.show(10)

+--------------------+--------------------+
|              Symbol|              values|
+--------------------+--------------------+
|FRED_00XALCCHM086...|1.679747251648322...|
|FRED_00XALCCHM086...| 1.67946514395112E-4|
|FRED_00XALCCHM086...|1.679183130998129...|
|FRED_00XALCCHM086...|1.678901212738281...|
|FRED_00XALCCHM086...|1.678619389124946...|
|FRED_00XALCCHM086...|1.678337660111495...|
|FRED_00XALCCHM086...|1.678056025649077...|
|FRED_00XALCCHM086...|1.677774485691063...|
|FRED_00XALCCHM086...|1.677493040188604...|
|FRED_00XALCCHM086...|1.677211689095070...|
+--------------------+--------------------+
only showing top 10 rows



In [63]:
from pyspark.sql.functions import monotonicallyIncreasingId

# This will return a new DF with all the columns + id
res = ts_exploded.withColumn("index", monotonicallyIncreasingId())

In [64]:
res.show(10)

+--------------------+--------------------+-----+
|              Symbol|              values|index|
+--------------------+--------------------+-----+
|FRED_00XALCCHM086...|1.679747251648322...|    0|
|FRED_00XALCCHM086...| 1.67946514395112E-4|    1|
|FRED_00XALCCHM086...|1.679183130998129...|    2|
|FRED_00XALCCHM086...|1.678901212738281...|    3|
|FRED_00XALCCHM086...|1.678619389124946...|    4|
|FRED_00XALCCHM086...|1.678337660111495...|    5|
|FRED_00XALCCHM086...|1.678056025649077...|    6|
|FRED_00XALCCHM086...|1.677774485691063...|    7|
|FRED_00XALCCHM086...|1.677493040188604...|    8|
|FRED_00XALCCHM086...|1.677211689095070...|    9|
+--------------------+--------------------+-----+
only showing top 10 rows



In [65]:
pivoted = res.groupBy("index").pivot("Symbol").sum("values")

In [71]:
from pyspark.sql.window import Window
ranked = res.select("Symbol", "index", "values",
     f.rowNumber()
     .over(Window
           .partitionBy("Symbol")
           .orderBy(f.col("index").desc())
            )
     .alias("rank")
    )

AnalysisException: u"Could not resolve window function 'row_number'. Note that, using window functions currently requires a HiveContext;"

In [68]:
pivoted.select(["index","FRED_00XALCCHM086NEST","FRED_00XALCCHM086NEST_shift"]).show()

+-----+---------------------+---------------------------+
|index|FRED_00XALCCHM086NEST|FRED_00XALCCHM086NEST_shift|
+-----+---------------------+---------------------------+
|   31| -8.67723575407319...|                       null|
|  231| 1.333929155022950...|                       null|
|  431| 4.685502386259088E-5|                       null|
|  631|                 null|                       null|
|  831|                 null|                       null|
| 1031|                 null|                       null|
| 1231|                 null|                       null|
| 1431|                 null|                       null|
| 1631|                 null|                       null|
| 1831|                 null|                       null|
| 2031|                 null|                       null|
| 2231|                 null|                       null|
| 2431|                 null|                       null|
| 2631|                 null|                       null|
| 2831|       

In [334]:
import datetime
import pandas as pd
import pandas.io.data
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
#from sklearn.ensemble import RandomForestClassifier
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)


In [335]:
start = datetime.datetime(2008, 1, 1)
end = datetime.datetime(2014, 8, 15)
#trim_start = '2000-01-01'
#trim_end = '2014-08-15'


In [336]:
sp =  pd.io.data.get_data_yahoo('^GSPC', start, end)
#sp.head(10)

In [337]:
sp.columns.values[-1] = 'AdjClose'
sp.columns = sp.columns + '_SP500'
sp['Return_SP500'] = sp['AdjClose_SP500'].pct_change()
sp.columns

Index([u'Open_SP500', u'High_SP500', u'Low_SP500', u'Close_SP500',
       u'Volume_SP500', u'AdjClose_SP500', u'Return_SP500'],
      dtype='object')

,Open_SP500,High_SP500,Low_SP500,Close_SP500,Volume_SP500,AdjClose_SP500,Return_SP500
Date,,,,,,,
2008-01-02,1467.969971,1471.770020,1442.069946,1447.160034,3452650000,1447.160034,NaN
2008-01-03,1447.550049,1456.800049,1443.729980,1447.160034,3429500000,1447.160034,0.000000
2008-01-04,1444.010010,1444.010010,1411.189941,1411.630005,4166000000,1411.630005,-0.024552
2008-01-07,1414.069946,1423.869995,1403.449951,1416.180054,4221260000,1416.180054,0.003223
2008-01-08,1415.709961,1430.280029,1388.300049,1390.189941,4705390000,1390.189941,-0.018352


In [339]:
nasdaq =  pd.io.data.get_data_yahoo('^IXIC', start, end)
#nasdaq.head()

In [340]:
nasdaq.columns.values[-1] = 'AdjClose'
nasdaq.columns = nasdaq.columns + '_Nasdaq'
nasdaq['Return_Nasdaq'] = nasdaq['AdjClose_Nasdaq'].pct_change()
nasdaq.columns

Index([u'Open_Nasdaq', u'High_Nasdaq', u'Low_Nasdaq', u'Close_Nasdaq',
       u'Volume_Nasdaq', u'AdjClose_Nasdaq', u'Return_Nasdaq'],
      dtype='object')

In [341]:
treasury =  pd.io.data.get_data_yahoo('^FVX', start, end)


In [342]:
treasury.columns.values[-1] = 'AdjClose'
treasury.columns = treasury.columns + '_Treasury'
treasury['Return_Treasury'] = treasury['AdjClose_Treasury'].pct_change()
treasury.columns

Index([u'Open_Treasury', u'High_Treasury', u'Low_Treasury', u'Close_Treasury',
       u'Volume_Treasury', u'AdjClose_Treasury', u'Return_Treasury'],
      dtype='object')

In [343]:
datasets = [sp, nasdaq, treasury]

In [344]:

to_be_merged = [nasdaq[['Return_Nasdaq']],
                treasury[['Return_Treasury']],
]
                
finance = sp[['Return_SP500']].join(to_be_merged, how = 'outer')

In [345]:
finance.head()

,Return_SP500,Return_Nasdaq,Return_Treasury
Date,,,
2008-01-02,NaN,NaN,NaN
2008-01-03,0.000000,-0.002663,-0.002140
2008-01-04,-0.024552,-0.037665,-0.029105
2008-01-07,0.003223,-0.002072,-0.003787
2008-01-08,-0.018352,-0.023585,0.000000
